<a href="https://colab.research.google.com/github/MayaHayat/EconAlgo_Ex5.3/blob/main/Ex5_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math
import doctest

# Part A

Create a graph with #players nodes, where each directed edge respresents the minimum valuation ratio of the taken items.

In [ ]:
def create_exchange_graph(valuations: list[list[float]], allocation: list[list[float]]):
    """
    Create a directed graph representing the exchange relationships between players based on valuations and allocation.

    Parameters:
    - valuations (list[list[float]]): List of lists representing valuations of players for each resource.
    - allocation (list[list[float]]): List of lists representing the allocation of resources to players.

    Returns:
    - nx.DiGraph: Directed graph representing exchange relationships between players.

    Examples:
    >>> valuations = [[10, 20, 30, 40], [40, 30, 20, 10]]
    >>> allocation = [[0, 0.7, 1, 1], [1, 0.3, 0, 0]]
    >>> graph = create_exchange_graph(valuations, allocation)
    >>> isinstance(graph, nx.DiGraph)
    True
    >>> graph.edges(data=True)
    OutEdgeDataView([('Player_0', 'Player_1', {'weight': 0.6666666666666666}), ('Player_1', 'Player_0', {'weight': 1.5})])

    >>> valuations2 = [[3, 6, 1], [6, 1, 3], [1, 3, 6]]
    >>> allocation2 = [[1, 0, 0], [0, 0, 1], [0, 1, 0]]
    >>> graph2 = create_exchange_graph(valuations2, allocation2)
    >>> isinstance(graph2, nx.DiGraph)
    True
    >>> graph2.edges(data=True)
    OutEdgeDataView([('Player_0', 'Player_1', {'weight': 0.5}), ('Player_0', 'Player_2', {'weight': 3.0}), ('Player_1', 'Player_0', {'weight': 3.0}), ('Player_1', 'Player_2', {'weight': 0.5}), ('Player_2', 'Player_0', {'weight': 0.5}), ('Player_2', 'Player_1', {'weight': 3.0})])
    """


    G = nx.DiGraph()

    num_players = len(valuations)
    num_resources = len(valuations[0])

    # Create nodes for players
    for i in range(num_players):
        G.add_node(f"Player_{i}")

    # Calculate and add weighted edges between players
    for i in range(num_players):
        for j in range(i+1, num_players):
            min_valuation_ratio_i = min(valuations[i][k] / valuations[j][k] for k in range(num_resources) if allocation[i][k] > 0)
            min_valuation_ratio_j = min(valuations[j][k] / valuations[i][k] for k in range(num_resources) if allocation[j][k] > 0)
            G.add_edge(f"Player_{i}", f"Player_{j}", weight=min_valuation_ratio_i)
            G.add_edge(f"Player_{j}", f"Player_{i}", weight= min_valuation_ratio_j)


    return G


Make sure there are no cycles with negative edges product.

In [ ]:
def is_pareto_efficient(valuations, allocation):
    """
    Check if an allocation is Pareto efficient using the exchange graph.

    Args:
    - valuations (list[list[float]]): List of valuations for each player.
    - allocation (list[list[float]]): Current allocation matrix.

    Returns:
    - bool: True if the allocation is Pareto efficient, False otherwise.

    Examples:
    >>> is_pareto_efficient([[10, 20, 30, 40], [40, 30, 20, 10]], [[0, 0.7, 1, 1], [1, 0.3, 0, 0]])
    True

    >>> is_pareto_efficient([[10, 20, 30, 40], [40, 30, 20, 10]], [[0.5, 1, 0, 1], [0.5, 0, 1, 0]])
    False

    >>> is_pareto_efficient([[3,6,1],[6,1,3],[1,3,6]], [[0,1,0],[1,0,0],[0,0,1]])
    True

    >>> is_pareto_efficient([[3,6,1],[6,1,3],[1,3,6]], [[1,0,0],[0,0,1],[0,1,0]])
    False
    """

    if all(all(alloc == 0 for alloc in player_alloc) for player_alloc in allocation):
        return False

    G = create_exchange_graph(valuations, allocation)
    # Convert edge weights to logarithmic values
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = math.log(G[edge[0]][edge[1]]['weight'])

    # Check for negative cycles using Bellman-Ford algorithm
    negative_cycle = nx.negative_edge_cycle(G, weight = 'weight')
    return not negative_cycle

# Part B

In [ ]:
def improve_pareto(valuations: list[list[float]], allocation: list[list[float]]):

    """
    Improve a given allocation to achieve Pareto efficiency.

    Args:
    - valuations (list[list[float]]): List of valuations for each player.
    - allocation (list[list[float]]): Current allocation matrix.

    Returns:
    - list[list[float]]: Improved Pareto efficient allocation.

    Examples:
    >>> improve_pareto([[10, 20, 30, 40], [40, 30, 20, 10]], [[0.5, 1, 0, 1], [0.5, 0, 1, 0]])
    [[0.0, 0.5, 1.0, 1], [1.0, 0.5, 0.0, 0]]

    >>> improve_pareto([[10, 20, 30, 40], [40, 30, 20, 10]], [[0, 0.7, 1, 1], [1, 0.3, 0, 0]])
    [[0, 0.7, 1, 1], [1, 0.3, 0, 0]]

    >>> improve_pareto([[10, 20, 30, 40], [40, 30, 20, 10]], [[0, 0.5, 1, 1], [1, 0.5, 0, 0]])
    [[0, 0.5, 1, 1], [1, 0.5, 0, 0]]

    >>> improve_pareto([[3, 6, 1], [6, 1, 3], [1, 3, 6]], [[0, 1, 0], [1, 0, 0], [0, 0, 1]])
    [[0, 1, 0], [1, 0, 0], [0, 0, 1]]

    >>> improve_pareto([[5, 10, 15], [15, 5, 10]], [[0.2, 1, 0], [0.8, 0, 1]])
    [[0.0, 1, 0.2], [1.0, 0, 0.8]]

    >>> improve_pareto([[8, 12], [12, 8]], [[1, 0],[0, 1]])
    [[0.0, 1.0], [1.0, 0.0]]
    """


    num_players = len(valuations)
    num_resources = len(valuations[0])

    improved_allocation = [row.copy() for row in allocation]
    epsilon = 0.1

    while not is_pareto_efficient(valuations, improved_allocation):
        current = create_exchange_graph(valuations, improved_allocation)

        # Iterate through each player
        for i in range(num_players):
            positive_allocation_resources = [k for k in range(num_resources) if improved_allocation[i][k] >= epsilon]

            # If there are positive allocation resources, choose the one with the minimum valuation
            if positive_allocation_resources:
                min_valuation_resource = min(positive_allocation_resources, key=lambda k: valuations[i][k])

            # Transfer a small portion (ε) of the minimum valuation resource to the next player
            improved_allocation[i][min_valuation_resource] = round(improved_allocation[i][min_valuation_resource] - epsilon, 2)
            improved_allocation[(i + 1) % num_players][min_valuation_resource] = round(improved_allocation[(i + 1) % num_players][min_valuation_resource] + epsilon, 2)

    return improved_allocation


In [ ]:
if __name__ == "__main__":
  doctest.testmod()